# Imports/Setup

The cell below provides imports all the libraries and packages needed to run this notebook. Additionally, it defines a number of global variables used repeatedly throughout. 

In [ ]:
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap
from qiskit.primitives import StatevectorSampler, Sampler
from qiskit_machine_learning.algorithms import QSVC, PegasosQSVC
from qiskit_aer import Aer


from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.utils import algorithm_globals

from sklearn.utils.class_weight import compute_sample_weight
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PowerTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, make_scorer, confusion_matrix, precision_score, recall_score
from sklearn.inspection import permutation_importance

from scipy.stats import shapiro, normaltest, boxcox, spearmanr, kendalltau
from scipy.special import boxcox1p

from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier


fig_count = 0
# !rm ../documents/figures/fig*

figsize_x = 6
figsize_y = 6

algorithm_globals.random_seed = 19
rs = 19

sig = 0.05

# Import Data

The cell below reads in the data:

In [ ]:
df = pd.read_csv('../data/creditcard_2023.csv', usecols=lambda column: column != 'id')
#df = df.sample(n=200, random_state=rs)
df.head()

# Dataset Summary

Provide a summary of the data:

In [ ]:
df.info()

The output elucidates that `df` contains 568,630 observations of 29 predictor variables (`V1`-`V28`, and `Class`) and one target variable, `Class`.

# Data Evaluation
## Class Imbalance

The cell below compares the observations in each class using histograms:

In [ ]:
fig_count += 1
plt.figure(figsize=(figsize_x, figsize_y))

class_counts = df['Class'].value_counts()

class_counts.plot(kind='bar')  # or use 'hist' for a histogram

for i, value in enumerate(class_counts.values):
    plt.text(i, value + 0.1, f'{value:,}', ha='center', va='bottom')
    
plt.xlabel('Class', fontweight='bold')
plt.ylabel('Count of Transactions',fontweight='bold')
plt.title('Transaction Count by Class', fontweight='bold')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

plt.grid(axis='y')
#plt.savefig(f'../documents/figures/fig_{fig_count}.png')
plt.show()

To make the dataset more realistic, the cell below drops drop positive (A.K.A fraudulent, 1) observations until they only make up 1% of the entire dataset. Removals are done at random.

In [ ]:
# fraud_prop = 0.01
# fraud_size = int((fraud_prop * class_counts[0]) / (1 - fraud_prop))
# num_removals = class_counts[1] - fraud_size

# condition = df['Class'] == 1
# rows_to_remove = df[condition].sample(n=num_removals, random_state=rs)
# df = df.drop(rows_to_remove.index)

In [ ]:
n = 10000
fraud_prop = 0.01
fraud_size = int(n * fraud_prop)
true_size = int(n * (1-fraud_prop))

df_pt1 = df[df['Class'] == 1].sample(n=fraud_size, random_state=rs)
df_pt2 = df[df['Class'] == 0].sample(n=true_size, random_state=rs)

df = pd.concat([df_pt1, df_pt2], ignore_index=True)

The plot below checks to ensure an appropriate number of observations have been removed:

In [ ]:
fig_count += 1
plt.figure(figsize=(figsize_x, figsize_y))

class_counts = df['Class'].value_counts()

class_counts.plot(kind='bar')  # or use 'hist' for a histogram

for i, value in enumerate(class_counts.values):
    plt.text(i, value + 0.1, f'{value:,}', ha='center', va='bottom')
    
plt.xlabel('Class', fontweight='bold')
plt.ylabel('Count of Transactions',fontweight='bold')
plt.title('Transaction Count by Class', fontweight='bold')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

plt.grid(axis='y')
#plt.savefig(f'../documents/figures/fig_{fig_count}.png')
plt.show()

Before moving onto the next section, the cell below splits `df` into two new dataframes: 

* `df_X`: Contains only the predictors present in `df`.
* `df_y`: Contains only the target variable in `df`: `Class`.

In [ ]:
df_X = df.drop(['Class'], axis=1)
df_y = df['Class']

## Outliers

Evaluating the presence of outliers is crucial to deciding the data pre-processing steps required for modelling. The cell below creates function that identifies the outliers in each column of a dataframe. 

In [ ]:
def identify_outliers(df):
    outlier_rows = []
    for column in df.select_dtypes(include='number'):  # Only process numeric columns
        Q1 = df[column].quantile(0.25)  # First quartile
        Q3 = df[column].quantile(0.75)  # Third quartile
        IQR = Q3 - Q1  # Interquartile range
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR


        for i, value in enumerate(df[column].values):
            if value < lower_bound or value > upper_bound:
                outlier_rows.append([column, i, value])
            
    # Get unique rows with at least one outlier
    return pd.DataFrame(outlier_rows, columns = ['column_name', 'row_num', 'value'])

The `identify_outliers` function is used below to identify all outliers in `df`:

In [ ]:
outlier_df = identify_outliers(df_X)
outlier_df = (
    outlier_df[['column_name', 'row_num']]
    .groupby(by='column_name').count()
    .sort_values(by='row_num')
    .reset_index()
)
outlier_df.columns = ['column_name', 'num_outliers']

The data contained within `outlier_df` is presented in the plot below:

In [ ]:
fig_count += 1
plt.figure(figsize=(10,6))

plt.bar(outlier_df['column_name'], outlier_df['num_outliers'])
label_push = outlier_df['num_outliers'].mean() * 0.01

for i, value in enumerate(outlier_df['num_outliers'].values):
    
    perc_outlier = (value / df.shape[0]) * 100
    plt.text(i, value + label_push, f'{perc_outlier:.1f}%', ha='center', va='bottom', fontsize=6)
    
plt.tick_params(axis='x', which='major', labelsize=8)
plt.xlabel('Variable Name', fontweight='bold')
plt.ylabel('Number of Outliers',fontweight='bold')
plt.title('Counts and Percentages of Outliers By Predictor', fontweight='bold')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

label_height = outlier_df['num_outliers'].max() * .90
plt.text(0, label_height, '* Percentage values represent the percentage of observations of each field that are outliers.', fontsize=8)

plt.grid(axis='y')
# #plt.savefig(f'../documents/figures/fig_{fig_count}.png')
# plt.show()

Clearly, the dataframe contains a reasonable number of outliers in many of the fields. This will be one of the primary considerations when choosing the data pre-processing steps in the next section. 

## Distributions

The shape of the distributions of each predictor is another consideration when determining the appropriate data pre-processing steps. They are plotted below using a boxplot:

In [ ]:
fig_count += 1
plt.figure(figsize=(10,6))
plt_df = df.drop(['Class', 'Amount'], axis=1)

sns.boxplot(data=plt_df)
plt.grid()
plt.xlabel('Variable Name', fontweight='bold')
plt.ylabel('Value', fontweight='bold')
plt.title('Distributions of Fields V1-V28', fontweight='bold')
plt.tick_params(axis='x', which='major', labelsize=8)
#plt.savefig(f'../documents/figures/fig_{fig_count}.png')
plt.show()

The many outliers make the distributions hard to see, but they can also be evaluated quantitatively to determine their shape. For instance, the cell below defines a function `evaluate_normality` that determines if each predictor variable exhibits a normal distribution using `scipy`'s `normaltest`:

In [ ]:
def evaluate_normality(df):
    results = []
    for column in df.columns:
        stat, p = normaltest(df[column])
        normal = p > 0.05
        results.append([column, stat, p, normal])
                       

    return pd.DataFrame(results, columns = ['variable_name', 'statistic', 'p_value', 'normal'])

In [ ]:
evaluate_normality(df_X)

The output above makes clear that none of the predictor variables can be considered to have a normal distribution. 

## Multicollinearity

The last aspect of the data taken into consideration before apply any pre-processing transformation is its multicollinearity. The cell below produces a correlation matrix of all the predictor fields, which checks the correlation of all pairs of predictors.

In [ ]:
correlation_matrix = df_X.corr()
sns.heatmap(correlation_matrix, cmap='coolwarm', fmt=".2f", square=True)
print(max(correlation_matrix))

The minimum and maximum correlation values are also displayed below: 

In [ ]:
correlation_matrix = correlation_matrix.replace(1, 0)

max_corr_value = correlation_matrix.select_dtypes(include='number').max().max()
min_corr_value = correlation_matrix.select_dtypes(include='number').min().min()

print(f'Minimum correlation value: {min_corr_value}')
print(f'Maximum correlation value: {max_corr_value}')

The output above coupled with the results of the correlation matrix provide strong evidence that no pair of predictor variables maintain a troublesome degree of correlation. 

# Data Pre-Processing

Based on the results of the previous section, there are two facets of the data that need to be addressed:

1. A large number of outliers present within many predictors.
2. Non-normal distributions for all predictors. 

There is no reason to believe the outliers are due to error, and should likely continue to be included in the dataset. However, given their quantity and scale, these outliers can have their influence limited by applying Winsorization, in which outliers are "pulled in" by capping extreme values to a specified percentile range. 

To address the non-normal (skewed) distributions of the predictors, we can perform a power transformation that will have the effect of smoothing the variance over the range of the distribution (reducing heteroscedasticity). This kind of transformation typically supports normalization. Box-Cox and Yeo-Johnson transformations are the most common approaches, but the latter will be applied in this case since the former only works for values $>0$. 

To prevent data leakage, these transformations will be fitted exclusively on a training dataset, and the resultant parameters will then be applied to transform the data.

First the cell below splits the data into testing and training matrices:

In [ ]:
X = df_X.values
y = df_y.values
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=rs)

Next, the cell below defines a function that can apply the windsorization transformation:

In [ ]:
def apply_winsorization(train_matrix, test_matrix, limits=(0.05, 0.95)):
    """
    Apply Winsorization to all columns in training and test matrices.

    Args:
        train_matrix (np.ndarray): Training data matrix.
        test_matrix (np.ndarray): Test data matrix.
        limits (tuple): Quantile limits for Winsorization (default is (0.05, 0.95)).

    Returns:
        np.ndarray, np.ndarray: Transformed training and test matrices.
    """
    train_matrix = train_matrix.copy()
    test_matrix = test_matrix.copy()

    for col in range(train_matrix.shape[1]):  # Loop over all columns
        lower, upper = np.quantile(train_matrix[:, col], limits)
        train_matrix[:, col] = np.clip(train_matrix[:, col], lower, upper)
        test_matrix[:, col] = np.clip(test_matrix[:, col], lower, upper)
    
    return train_matrix, test_matrix

The training and testing data are transformed below using the `apply_windsorization` function:

In [ ]:
X_train, X_test = apply_winsorization(X_train, X_test)

Next, the cell below applies the Yeo-Johnson transformation:

In [ ]:
yj = PowerTransformer(method='yeo-johnson')

X_train = yj.fit_transform(X_train)
X_test = yj.transform(X_test)

The final pre-processing step is to scale the data appropriately. Given that the data was not originally considered normal, standard scaling (in which $\mu$=0 and $\sigma=1$ is not appropriate. Furthermore, min-max scaling (in which all fields are scaled to be between 0 and 1) is sensitive to outliers, which excludes it as a choice in this case. Robust scaling is a common choice when dealing with data with outliers, since it works by centering the data it each field around its median value and then scaling according to its IQR. As such, this was the scaling option implemented: 

In [ ]:
scaler = RobustScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
df_X_pp = pd.DataFrame(np.vstack((X_train, X_test)), columns=df_X.columns)
df_y_pp = pd.Series(np.concatenate([y_train, y_test]), name=df_y.name)
df_pp = pd.concat([df_X_pp, df_y_pp], axis=1)

The result of the scaling is shown in the following plot: 

In [ ]:
fig_count += 1

plt.figure(figsize=(10,6))

sns.boxplot(data=df_X_pp)
plt.grid()
plt.xlabel('Variable Name', fontweight='bold')
plt.ylabel('Value', fontweight='bold')
plt.title('Distributions of All Fields (Post Pre-Processing)', fontweight='bold')
plt.xticks(rotation=45)
#plt.savefig(f'../documents/figures/fig_{fig_count}.png')
plt.show()

It is clear that the result of the pre-processing steps resulted in a significantly more interpretable dataset. The ranges of each field are similar and outliers now only exist in two (instead of all) fields: 

In [ ]:
outlier_df = identify_outliers(df_X_pp)
outlier_df = (
    outlier_df[['column_name', 'row_num']]
    .groupby(by='column_name').count()
    .sort_values(by='row_num')
    .reset_index()
)
outlier_df.columns = ['column_name', 'num_outliers']
outlier_df

Unfortunately, the data is still not considered to be normal, meaning that further pre-processing transformations such as PCA would likely not be as effective. However, multicollinearity was deemed earlier not to be a significant concern in this dataset, meaning there is not likely a great need for dimensionality reduction regardless. 

In [ ]:
evaluate_normality(df_X_pp)

# Exploratory Data Analysis

The cell below evaluates which predictors might be most important to the model by performing $t$-tests of the means of each predictor variable separated by class. 

In [ ]:
df_class0 = df_pp[df_pp['Class'] == 0]
df_class1 = df_pp[df_pp['Class'] == 1]

p_vals = []
for col in df_X.columns:
    
    group1 = df_class0[col]
    group2 = df_class1[col]
    
    t_statistic, p_value = stats.ttest_ind(group1, group2, equal_var=False)
    
    p_vals.append(p_value)

The results of these t-tests can be visualized below:

In [ ]:
fig_count += 1
plt.figure(figsize=(10, figsize_y))

xs = np.arange(0,len(p_vals))
plt.scatter(xs, p_vals, label='$t$-test $p$-values')
plt.axhline(sig, color='black', ls='--', label='Significance Barrier')
plt.axhspan(0, sig, color='green', alpha=0.2)
plt.axhspan(sig, 1, color='red', alpha=0.2)
plt.xticks(xs, df_X.columns)
plt.xlabel('Variable Name', fontweight='bold')
plt.ylabel('$p$-value', fontweight='bold')
plt.title('$t$-test Results When Comparing the Means of Each Predictor By Class', fontweight='bold')
plt.ylim(-0.05,1)
plt.xticks(rotation=45)
plt.legend()
plt.grid()
#plt.savefig(f'../documents/figures/fig_{fig_count}.png')
plt.show()

Most of the variables exhibit statistically significant differences when comparing the means of the observations in each class. The cells below provide histograms of two of these predictors as examples, highlighting the difference in distribution when split by class. 

In [ ]:
fig_count += 1
plt.figure(figsize=(figsize_x, figsize_y))

group1 = df_class0['V1']
group2 = df_class1['V1']

plt.hist(group1, bins=15, alpha=0.5, density=True, label='Class 0')
plt.hist(group2, bins=15, alpha=0.5, density=True, label='Class 1')

plt.title('Distribution Comparison of V1 Feature', fontweight='bold')
plt.grid()
plt.xlabel('Value', fontweight='bold')
plt.ylabel('Probability Density', fontweight='bold')
#plt.savefig(f'../documents/figures/fig_{fig_count}.png')
plt.show()

In [ ]:
fig_count += 1
plt.figure(figsize=(figsize_x, figsize_y))

group1 = df_class0['V4']
group2 = df_class1['V4']

plt.hist(group1, bins=15, alpha=0.5, density=True, label='Class 0')
plt.hist(group2, bins=15, alpha=0.5, density=True, label='Class 1')

plt.title('Distribution Comparison of V4 Feature', fontweight='bold')
plt.grid()
plt.xlabel('Value', fontweight='bold')
plt.ylabel('Probability Density', fontweight='bold')
plt.show()

## PCA

In [ ]:
pca = PCA()
pca.fit(X_train)

explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)
n_components_to_keep = np.argmax(cumulative_variance >= 0.95) + 1

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(explained_variance) + 1), cumulative_variance, marker='o', linestyle='--', label='Cumulative Explained Variance')
plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.6, label='Individual Explained Variance')

plt.axhline(y=0.95, color='r', linestyle='--', label='95% Variance Threshold')
plt.title('Explained Variance vs. Number of Components', fontweight='bold')
plt.xlabel('Number of Components', fontweight='bold')
plt.ylabel('Explained Variance', fontweight='bold')
plt.xticks(range(1, len(explained_variance) + 1))
plt.legend(loc='best')
plt.grid()
plt.show()

In [ ]:
# pca = PCA(n_components = n_components_to_keep)
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)

In [ ]:
# fig_count += 1
# plt.figure(figsize=(figsize_x, figsize_y))

# class_counts = pd.Series(y_train).value_counts()

# class_counts.plot(kind='bar')  # or use 'hist' for a histogram

# for i, value in enumerate(class_counts.values):
#     plt.text(i, value + 0.1, f'{value:,}', ha='center', va='bottom')
    
# plt.xlabel('Class', fontweight='bold')
# plt.ylabel('Count of Transactions',fontweight='bold')
# plt.title('Transaction Count by Class', fontweight='bold')
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

# plt.grid(axis='y')
# #plt.savefig(f'../documents/figures/fig_{fig_count}.png')
# plt.show()

# Classical Machine Learning 
## SVM


In [ ]:
svm = SVC(class_weight='balanced', random_state=rs) 

# Define the hyperparameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [1, 0.1, 0.01],
    'kernel': ['linear', 'rbf']
}

# Perform grid search
scoring_fcn = make_scorer(f1_score, pos_label=1)
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=3, scoring=scoring_fcn, verbose=2)
grid_search.fit(X_train, y_train)

# Print best parameters and evaluate on the test set
print("Best Parameters:", grid_search.best_params_)
best_svm_model = grid_search.best_estimator_
y_pred = best_svm_model.predict(X_test)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
result = permutation_importance(best_svm_model, X_test, y_test, n_repeats=10)
importances = result.importances_mean
importance_df_svm = pd.DataFrame({'variable_name': df_X_pp.columns, 'permutation_importance': importances})
importance_df_svm = importance_df_svm.sort_values(by='permutation_importance', ascending=False).reset_index()

In [ ]:
plt.figure(figsize=(10,figsize_y))

plt.bar(importance_df_svm['variable_name'], importance_df_svm['permutation_importance'])

plt.title('Permutation Importance (Classical SVM Model)', fontweight='bold')
plt.xlabel('Variable Name', fontweight='bold')
plt.ylabel('Permutation Importance', fontweight='bold')
plt.xticks(rotation=45, fontsize = 7)

plt.grid()
plt.show()

## Random Forest

In [ ]:
rf = RandomForestClassifier(class_weight='balanced', random_state=rs)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [1, 2]
}

# Perform grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring=scoring_fcn, verbose=2)
grid_search.fit(X_train, y_train)

# Print best parameters and evaluate on the test set
print("Best Parameters:", grid_search.best_params_)
best_rf_model = grid_search.best_estimator_
y_pred = best_rf_model.predict(X_test)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
feature_importances = best_rf_model.feature_importances_

importance_df_rf = pd.DataFrame({'variable_name': df_X_pp.columns, 'feature_importance': feature_importances})
importance_df_rf = importance_df_rf.sort_values(by='feature_importance', ascending=False).reset_index(drop=True)

plt.figure(figsize=(10, figsize_y))
plt.bar(importance_df_rf['variable_name'], importance_df_rf['feature_importance'])
plt.title('Feature Importances (Classical RF Model)', fontweight='bold')
plt.xlabel('Variable Name', fontweight='bold')
plt.ylabel('Feature Importance', fontweight='bold')
plt.xticks(rotation=45)
plt.grid()
plt.show()

In [ ]:
feature_ranks = []

for column in df_X_pp.columns:
    rank_rf = importance_df_rf[importance_df_rf['variable_name'] == column].index.item()
    rank_svm = importance_df_svm[importance_df_svm['variable_name'] == column].index.item()
    feature_ranks.append([column, rank_rf, rank_svm])

feature_ranks_df = pd.DataFrame(feature_ranks, columns=['variable_name', 'rf_importance', 'svm_importance'])
rho, p_val = spearmanr(feature_ranks_df['rf_importance'], feature_ranks_df['svm_importance'])
tau, p_val = kendalltau(feature_ranks_df['rf_importance'], feature_ranks_df['svm_importance'])
rho

# Quantum Algorithm Evaluation
## Data Prep

In [ ]:
# n = 1000
# fraud_prop = 0.10
# fraud_size = int(n * fraud_prop)
# true_size = int(n * (1-fraud_prop))

# q_df_pt1 = df_pp[df_pp['Class'] == 1].sample(n=fraud_size, random_state=rs)
# q_df_pt2 = df_pp[df_pp['Class'] == 0].sample(n=true_size, random_state=rs)

# q_df = pd.concat([q_df_pt1, q_df_pt2], ignore_index=True)
# q_X = q_df.drop(columns='Class').values
# q_y = q_df['Class'].values

q_X_train, q_X_test, q_y_train, q_y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=rs)



smote = SMOTE(random_state=rs, k_neighbors=3)
q_X_train, q_y_train = smote.fit_resample(q_X_train, q_y_train)

q_train = pd.DataFrame(np.hstack((q_X_train, q_y_train.reshape(-1,1))), columns=df_pp.columns)
q_test = pd.DataFrame(np.hstack((q_X_test, q_y_test.reshape(-1,1))), columns=df_pp.columns)

n = 50
q_train_df_pt1 = q_train[q_train['Class'] == 1].sample(n=n, random_state=rs)
q_train_df_pt2 = q_train[q_train['Class'] == 0].sample(n=n, random_state=rs)

n = 25
q_test_df_pt1 = q_test[q_test['Class'] == 1].sample(n=n, random_state=rs)
q_test_df_pt2 = q_test[q_test['Class'] == 0].sample(n=n, random_state=rs)

q_train = pd.concat([q_train_df_pt1, q_train_df_pt2], ignore_index=True)
q_test = pd.concat([q_test_df_pt1, q_test_df_pt2], ignore_index=True)

q_X_train = q_train.drop(columns='Class').values
q_y_train = q_train['Class'].values
q_X_test = q_test.drop(columns='Class').values
q_y_test = q_test['Class'].values

## QSVC

In [ ]:
def fit_qsvc(X_train, y_train, reps, entanglement):
    
    feature_map = ZZFeatureMap(feature_dimension=X_train.shape[1], reps=reps, entanglement=entanglement)
    sampler = StatevectorSampler()
    fidelity = ComputeUncompute(sampler=sampler, num_virtual_qubits=X_train.shape[1])
    qkernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)
    qsvc = QSVC(quantum_kernel=qkernel)
    sample_weights = compute_sample_weight(class_weight="balanced", y=y_train)

    start_time = time.time()
    qsvc.fit(X_train, y_train, sample_weight=sample_weights) 
    end_time = time.time()
    execution_time = end_time - start_time
    
    return qsvc, execution_time

In [ ]:
# qsvc, runtime = fit_qsvc(X_train_tmp, q_y_train, 1, 'linear')
# y_pred = qsvc.predict(X_test_tmp)
# report = classification_report(q_y_test, y_pred)

In [ ]:
def evaluate_qsvc(qubits_range, entanglement_types, reps_range, importance_df, X_train, X_test, y_train, y_test):

    results = []
    for num_qubits in qubits_range:
            
        feature_names = importance_df['variable_name'].values[:num_qubits]
        feature_indices = df_X_pp.columns.get_indexer(feature_names)
        X_train_tmp = X_train[:, feature_indices]
        X_test_tmp = X_test[:, feature_indices]

        for entanglement_type in entanglement_types:
            for reps in reps_range:

                print('Running QSVC algorithm with the following parameters...')
                print(f'num_qubits={num_qubits}, entanglement_type={entanglement_type}, reps={reps}')
                
                qsvc, runtime = fit_qsvc(X_train_tmp, q_y_train, reps, entanglement_type) 
                y_pred =  qsvc.predict(X_test_tmp)
                f1_class1 = f1_score(y_test, y_pred, pos_label=1)
                f1_class0 = f1_score(y_test, y_pred, pos_label=0)
                precision_class1 = precision_score(y_test, y_pred, pos_label=1)
                precision_class0 = precision_score(y_test, y_pred, pos_label=0)
                recall_class1 = recall_score(y_test, y_pred, pos_label=1)
                recall_class0 = recall_score(y_test, y_pred, pos_label=0)
                accuracy = accuracy_score(y_test, y_pred)
    
                results.append([num_qubits, entanglement_type, reps, runtime, f1_class1, f1_class0, precision_class1,
                                precision_class0, recall_class1, recall_class0, accuracy])


    column_names = ['num_quibits', 'entanglement_type', 'reps', 'runtime', 'f1_class1', 'f1_class0', 'precision_class1',
                    'precision_class0', 'recall_class1', 'recall_class0', 'accuracy']
                
    return pd.DataFrame(results, columns=column_names)

In [ ]:
qubits_range = list(range(2,10))
entanglement_types = ['linear', 'circular', 'full']
reps_range = [1,2,3]

qsvc_results = evaluate_qsvc(qubits_range, 
                             entanglement_types, 
                             reps_range, 
                             importance_df_svm, 
                             q_X_train, q_X_test, q_y_train, q_y_test)

In [ ]:
qsvc_results

## Pegasos QSVC

In [ ]:
def fit_pegasos_qsvc(X_train, y_train, reps, C, tau):
    
    feature_map = ZFeatureMap(feature_dimension=q_X_train.shape[1], reps=1)
    qkernel = FidelityQuantumKernel(feature_map=feature_map)
    pegasos_qsvc = PegasosQSVC(quantum_kernel=qkernel, C=C, num_steps=tau)

    start_time = time.time()
    pegasos_qsvc.fit(q_X_train, y_train) 
    end_time = time.time()
    fit_time = end_time - start_time
    
    return pegasos_qsvc, execution_time

In [ ]:
pegasos_qsvc, runtime = fit_pegasos_qsvc(q_X_train, y_train, 1, 1000, 100)

In [ ]:
def evaluate_pegasos_qsvc(qubits_range, reps_range, C_range, tau_range, importance_df, X_train, X_test, y_train, y_test):

    results = []
    for num_qubits in qubits_range:
            
        feature_names = importance_df['variable_name'].values[:num_qubits]
        feature_indices = df_X_pp.columns.get_indexer(feature_names)
        X_train_tmp = X_train[:, feature_indices]
        X_test_tmp = X_test[:, feature_indices]

        for reps in reps_range:        
            for C in C_range:
                for tau in tau_range:

                    print('Running PegasosQSVC algorithm with the following parameters...')
                    print(f'num_qubits={num_qubits}, reps={reps}, C={C}, tau={tau}')
                
                    pegasos_qsvc, runtime = fit_pegasos_qsvc(X_train_tmp, q_y_train, C, tau) 
                    y_pred =  pegasos_qsvc.predict(X_test_tmp)
                    f1_class1 = f1_score(y_test, y_pred, pos_label=1)
                    f1_class0 = f1_score(y_test, y_pred, pos_label=0)
                    precision_class1 = precision_score(y_test, y_pred, pos_label=1)
                    precision_class0 = precision_score(y_test, y_pred, pos_label=0)
                    recall_class1 = recall_score(y_test, y_pred, pos_label=1)
                    recall_class0 = recall_score(y_test, y_pred, pos_label=0)
                    accuracy = accuracy_score(y_test, y_pred)
        
                    results.append([num_qubits, reps, C, tau, runtime, f1_class1, f1_class0, precision_class1,
                                    precision_class0, recall_class1, recall_class0, accuracy])


    column_names = ['num_quibits', 'reps', 'C', 'tau', 'runtime', 'f1_class1', 'f1_class0', 'precision_class1',
                    'precision_class0', 'recall_class1', 'recall_class0', 'accuracy']
                
    return pd.DataFrame(results, columns=column_names)